In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import html2text
import os


In [18]:
#helper function get first number in string

def str_first_num(s):
    first_num = ''
    flag_digit=False
    for el in s: 
        if el.isdigit():
            first_num += el
            flag_digit = True
        else:
            if flag_digit:
                break
    return first_num
    

In [3]:
#read question urls from csv file
src_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\quet_indexes.csv"
df = pd.read_csv(src_path)

In [4]:
#get list of urls
url_list = list(df['url'].values)

In [5]:
url_list

['https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/How-long-could-the-brain-survive-theoretically-if-we-had-the-technology-to-replace-all-other-body-parts-as-you-aged-with-functioning-organs-that-are-grown-in-a-lab-using-your-DNA',
 'https://www.quora.com/What-are-the-top-10-emerging-technologies-in-the-next-5-10-years-2020%E2%80%932025',
 'https://www.quora.com/What-are-the-upcoming-emerging-technologies-in-software-industry',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-most-advanced-technologies-that-people-dont-know-about-yet',
 'https://www.quora.com/When-will-Fusion-reactors-become-a-reality',
 'https://www.quora.com/In-future-IOT-Internet-of-things-is-trending-technology-or-not',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-solutions-to-emerging-issues-in-communication']

In [6]:
#request url html
page = requests.get(url_list[0])

#create soup object with htmal parser
soup = BeautifulSoup(page.content, 'html.parser')


In [7]:
answers = soup.find_all('script', {"type": "application/ld+json"})[0]
#convert soup object to string
answers = [str(a) for a in answers][0]
#split answers with text separator
answers_list = answers.split('"text":')

In [8]:
columns = ["Answer", "name", "knowsAbout",  "hasCredential", "upvoteCount", "affiliation", "answerCount", "followerCount"]

#create df with nans to fill it later with values
x_shape = (len(answers_list)-1, len(columns))
x = np.tile(np.nan, x_shape)
answers_df = pd.DataFrame(x, columns = columns)

In [9]:
#get answer value
split_post = answers_list[1].split("upvoteCount")
answers_df["Answer"].iloc[0] = split_post[0]

#remove answer
split_post = split_post[1]

#get upvote value
upvote = str_first_num(split_post)

answers_df["upvoteCount"].iloc[0] = float(upvote)

#get affiliation which is right before answercount
split_post = split_post.split("affiliation")
split_post = split_post[1]
split_post = split_post.split("answerCount")

answers_df["affiliation"].iloc[0] = split_post[0]

split_post = split_post[1]

#get answer count
answers_df["answerCount"].iloc[0] = str_first_num(split_post)

#get followercount
split_post = split_post.split("followerCount")
answers_df['followerCount'] .iloc[0]= str_first_num(split_post[1])

split_post = split_post[1]

#get person credentials (between hascredentials and knowsabout)
split_post = split_post.split("hasCredential")[1]
split_post = split_post.split("knowsAbout")

#remove url from credential part
answers_df['hasCredential'].iloc[0] = split_post[0].split("image")[0]
split_post = split_post[1]

#Finally, get knowsabout and name. Remove url image from knowsabout too
split_post = split_post.split("name")
answers_df['knowsAbout'].iloc[0] = split_post[0].split("image")[0]

answers_df['name'].iloc[0] = split_post[1]


 

C:\Users\Ravit\AppData\Local\Temp\ipykernel_5296\864728059.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df["upvoteCount"].iloc[0] = float(upvote)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_5296\864728059.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df["affiliation"].iloc[0] = split_post[0]
C:\Users\Ravit\AppData\Local\Temp\ipykernel_5296\864728059.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans

In [10]:
answers_list[1]

' "50 years is a very long time. The way I make such predictions is by looking at technology that works in the lab today plus dreams that are achievable with a few scientific breakthroughs. Here are some ideas.\\n\\nLong distance travel will be by hyperloop or aircraft/rocket. Hyperloop is very efficient and fast, but limited to fixed routes. Earth- and space-bound craft will use similar propulsion (see below), and the main differences will be details like whether things are designed for low gravity.\\n\\nWe\'ll have autonomous personal multirotor aircraft (drones). We have the technology today but the energy costs are extreme.\\n\\nCheap clean energy changes everything and simply will happen. There are too many routes to it and everyone wants it (except the Saudis of course). Solar, thorium reactors, improved uranium reactors, and of course fusion could all play a role, especially in the next 30 years. But more likely some new technology will be invented. We know of plenty of ways to 

In [19]:
def answers_into_df(url):
    '''input: url for scrapping answers to a particular question.
        out: df with answers for each url/question'''
    
    #request url html
    page = requests.get(url)
   
    #create soup object with htmal parser
    soup = BeautifulSoup(page.content, 'html.parser')

    answers = soup.find_all('script', {"type": "application/ld+json"})[0]
    #convert soup object to string
    answers = [str(a) for a in answers][0]
    #split answers with text separator
    answers_list = answers.split('"text":')
    #first element is just the header of a page. remove it
    answers_list = answers_list[1:]
    
    #create df with nans to fill it later with values
    columns = ["Answer", "name", "knowsAbout",  "hasCredential", "upvoteCount", "affiliation", "answerCount", "followerCount"]
    x_shape = (len(answers_list)-1, len(columns))
    x = np.tile(np.nan, x_shape)
    answers_df = pd.DataFrame(x, columns = columns)

    for i, ans in enumerate(answers_list):    
        #get answer value
        split_post = ans.split("upvoteCount")
        answers_df["Answer"].iloc[i] = split_post[0]

        #remove answer
        split_post = split_post[1]

        #get upvote value
        upvote = str_first_num(split_post)

        answers_df["upvoteCount"].iloc[i] = float(upvote)

        #get affiliation which is right before answercount
        if 'affiliation' in ans:    
            split_post = split_post.split("affiliation")
            split_post = split_post[1]
            split_post = split_post.split("answerCount")

        answers_df["affiliation"].iloc[i] = split_post[0]

        split_post = split_post[1]

        #get answer count
        answers_df["answerCount"].iloc[i] = str_first_num(split_post)

        #get followercount
        split_post = split_post.split("followerCount")
        answers_df['followerCount'] .iloc[i]= str_first_num(split_post[1])

        split_post = split_post[1]

        #get person credentials (between hascredentials and knowsabout)
        split_post = split_post.split("hasCredential")[1]
        split_post = split_post.split("knowsAbout")

        #remove url from credential part
        answers_df['hasCredential'].iloc[i] = split_post[0].split("image")[0]
        split_post = split_post[1]

        #Finally, get knowsabout and name. Remove url image from knowsabout too
        split_post = split_post.split("name")
        answers_df['knowsAbout'].iloc[i] = split_post[0].split("image")[0]

        answers_df['name'].iloc[i] = split_post[1]

    return answers_df   
    

In [20]:

answers_into_df(url_list[0])

C:\Users\Ravit\AppData\Local\Temp\ipykernel_5296\4121122546.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df["upvoteCount"].iloc[i] = float(upvote)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_5296\4121122546.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answers_df["affiliation"].iloc[i] = split_post[0]
C:\Users\Ravit\AppData\Local\Temp\ipykernel_5296\4121122546.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

IndexError: list index out of range